In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import matplotlib.pyplot as plt

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}

def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df['geometry'] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis='columns'), geometry='geometry')

In [39]:
gdf = pd.read_csv('./data/air_quality/2023/2023_tramer_no2_mapa_qualitat_aire_bcn.csv')

gdf = convert_wkt_to_geometry(gdf, 'GEOM_WKT')

gdf['Rang'] = gdf['Rang'].astype('category')

In [ ]:
px.histogram(
    gdf,
    x="Rang",
    title="Histograma de la qualitat de l'aire a Barcelona",
    labels={"Rang": "Qualitat de l'aire"},
    category_orders={"Rang": gdf["Rang"].cat.categories},
)

In [ ]:
plt.figure(figsize=(6, 10))
gdf.plot(
    ax=plt.gca(),
    column="Rang",
    legend=True
)

---

In [2]:
df = pd.concat(
    [
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_1S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_2S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
    ]
)

df["date"] = pd.to_datetime(
    df["Any"].astype(str)
    + "-"
    + df["Mes"].astype(str)
    + "-"
    + df["Dia"].astype(str)
    + " "
    + df["Hora"].astype(str)
)

df = df.drop(columns=["Any", "Mes", "Dia", "Hora"])

gdf = pd.read_csv("./data/noise_monitoring/XarxaSoroll_EquipsMonitor_Instal.csv")
gdf = gdf[gdf["Id_Instal"].isin(df["Id_Instal"].unique())]

In [ ]:
px.line(
    df,
    x="date",
    y="Nivell_LAeq_1h",
    color="Id_Instal",
    title="Nivell de soroll a Barcelona",
)

In [ ]:
fig = px.scatter_mapbox(
    gdf,
    lat="Latitud",
    lon="Longitud",
    hover_name="Nom_Carrer",
    hover_data=["Nom_Barri", "Nom_Districte"],
    color="Font",
    zoom=10,
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.sunburst(
    df.merge(gdf[["Id_Instal", "Nom_Districte", "Font"]], on="Id_Instal"),
    path=["Nom_Districte", "Font"],
    title="Sunburst of noise type in Barcelona by district",
)
fig.show()

In [ ]:
px.histogram(
    data_frame=df,
    x="Nivell_LAeq_1h",
    marginal="box",
    nbins=250,
)

---

In [6]:
df = pd.read_csv("./data/district_zone/BarcelonaCiutat_Districtes.csv")

gdf = convert_wkt_to_geometry(df, 'geometria_wgs84')

In [ ]:
fig = px.choropleth_map(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="nom_districte",
    center=CENTER_BARCELONA,
    title="Map of Districts in Barcelona"
)

fig.show()